## Import libraries

In [1]:
import os

files = os.listdir('.')
print(files)

['shopping_list.json', 'Social_Media_Strategist_Agent.py', 'leftover.py', '.env', '__pycache__', 'test.py', 'CrewAI_1.ipynb', '.ipynb_checkpoints', 'Meal_&_GroceryPlanner.py', 'Course', 'requirements.txt', 'config', 'Meal & Grocery Planner.ipynb', 'meals.json']


In [2]:
from pydantic import BaseModel, Field
from typing import List, Dict, Optional, Literal
from IPython.display import display, JSON, Markdown
from datetime import datetime
from leftover import LeftoversCrew
import os

## Our Grocery Shopping Data Structure

1. ```GroceryItem```         → Individual grocery item with details
2. ```MealPlan```           → Recipe information with researched ingredients  
3. ```ShoppingCategory```   → Store section with organized items
4. ```GroceryShoppingPlan``` → Complete shopping strategy with budget analysis

In [3]:
class GroceryItem(BaseModel):
    """Individual grocery item"""
    name: str = Field(description="Name of the grocery item")
    quantity: str = Field(description="Quantity needed (for example, '2 kg', '200 gram')")
    estimated_price: str = Field(description="Estimated price (for example, '₹100-500')")
    category: str = Field(description="Store section (for example, 'Produce', 'Dairy')")

In [4]:
sample_item = GroceryItem(
    name="Kheer",
    quantity="250 gram",
    estimated_price="₹ 100",
    category="Dairy"
)

sample_item

GroceryItem(name='Kheer', quantity='250 gram', estimated_price='₹ 100', category='Dairy')

In [5]:
type(sample_item)

__main__.GroceryItem

In [6]:
# Display structured data
print("🛒 Sample Grocery Item Structure:")
display(JSON(sample_item.model_dump()))

🛒 Sample Grocery Item Structure:


<IPython.core.display.JSON object>

### MealPlan

It represents a complete meal with all the details needed for cooking, including:

- **`meal_name`** – The name of the dish being prepared (for example, "Rasgulla").
- **`taste`** – How challenging it is to cook ("Salty", "Sweet", "sour").
- **`servings`** – Number of people the meal will feed (integer value).
- **`researched_ingredients`** – List of ingredients found through AI research (note this is a list).


In [7]:
class MealPlan(BaseModel):
    """Simple meal plan"""
    meal_name: str = Field(description="Name of the meal")
    taste: str = Field(description="Taste profile of the meal 'Salty', 'Sweet', 'Sour'")
    servings: int = Field(description="Number of people it serves")
    researched_ingredients: List[str] = Field(description="Ingredients found through research")


In [8]:
# Corrected sample
sample_meal = MealPlan(
    meal_name="Samosa",
    taste="Salty",  # Now valid with Literal constraint
    servings=2,
    researched_ingredients=["Potato", "Paneer", "Peas", "Spices"]  # Completed the list example
)

In [9]:
print("\n🍽️ Sample Meal Plan Structure:")
display(JSON(sample_meal.model_dump()))


🍽️ Sample Meal Plan Structure:


<IPython.core.display.JSON object>

### ShoppingCategory

It organizes items by store layout for efficient shopping, containing:

- **`section_name`** – The store department name (for example, "Produce", "Meat & Poultry").
- **`items`** – A collection of GroceryItem objects in this section (note this is a list).
- **`estimated_total`** – The expected cost for all items in this category.


In [10]:
class ShoppingCategory(BaseModel):
    """Store section with items"""
    section_name: str = Field(description="Store section (for example, 'Produce', 'Dairy')")
    items: List[GroceryItem] = Field(description="Items in this section")
    estimated_total: str = Field(description="Estimated cost for this section in Indian Rupees (₹)")

In [11]:
produce_section = ShoppingCategory(
    section_name="Produce",
    items=[
        GroceryItem(
            name="Potatoes",
            quantity="2 kg",
            estimated_price="₹91-100",  # Range format as per description
            category="Produce"
        ),
        GroceryItem(
            name="Tomatoes",
            quantity="1 kg",
            estimated_price="₹55-65",
            category="Produce"
        )
    ],
    estimated_total="₹150-170"  # Approximate sum range for section
)

In [12]:
print("\n🏪 Sample Shopping Section:")
display(JSON(produce_section.model_dump()))


🏪 Sample Shopping Section:


<IPython.core.display.JSON object>

### GroceryShoppingPlan

It is the master plan that combines everything into a complete shopping strategy:

- **`total_budget`** – The overall spending limit for the shopping trip.
- **`meal_plans`** – Collection of meals being prepared (note this is a list).
- **`shopping_sections`** – Organized categories for store navigation (note this is a list).
- **`shopping_tips`** – Money-saving advice and practical suggestions (note this is a list).


In [13]:
class GroceryShoppingPlan(BaseModel):
    """Complete simplified shopping plan"""
    total_budget: str = Field(description="Total planned budget")
    meal_plans: List[MealPlan] = Field(description="Planned meals")
    shopping_sections: List[ShoppingCategory] = Field(description="Organized by store sections")
    shopping_tips: List[str] = Field(description="Money-saving and efficiency tips")

## Setting Up Our LLM and Essential Tools  

In [14]:
import os
from crewai_tools import SerperDevTool
from crewai import Agent, Task, Crew, Process
from crewai import LLM
from dotenv import load_dotenv

# Set environment variables
load_dotenv()

True

In [15]:
perplexity_llm = LLM(
    model="sonar-pro", 
    base_url="https://api.perplexity.ai",
    api_key=os.getenv("PERPLEXITY_API_KEY"),  # Required!
    temperature=0.3,
    max_tokens=500
)



# Local Ollama LLM fallback (if needed)
ollama_llm = LLM(
    model="ollama/llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.3,
    max_tokens=100
)

In [16]:
meal_planner = Agent(
    role="Meal Planner & Recipe Researcher",
    goal="Search for optimal recipes and create detailed meal plans",
    backstory="A skilled meal planner who researches the best recipes online, considering dietary needs, taste preferences, and budget constraints.",
    tools=[SerperDevTool()],
    llm=perplexity_llm,
    verbose=False)


In [17]:
meal_planning_task = Task(
    description=(
        "Search for the best '{meal_name}' recipe for {servings} people within a {budget} budget. "
        "Consider dietary restrictions: {dietary_restrictions} and Taste preference : {taste}. "
        "Find recipes that match the Taste preference and provide complete ingredient lists with quantities."
    ),
    expected_output="A detailed meal plan with researched ingredients, quantities, and cooking instructions appropriate for the skill level.",
    agent=meal_planner,
    output_pydantic=MealPlan,
    output_file="meals.json"
)

### Creating and Running Our Meal Planning Crew

In [18]:
from crewai import Crew, Process

meal_planner_crew = Crew(
    agents=[meal_planner],
    tasks=[meal_planning_task],
    process=Process.sequential,  # Ensures tasks are executed in order
    verbose=True
)

meal_planner_result = meal_planner_crew.kickoff(
    inputs={
        "meal_name": "Kheer",
        "servings": 4,
        "budget": "₹25",                           
        "dietary_restrictions": ["no nuts"],       
        "taste": "Sweet"                
    }
)
print("✅ Single meal planning completed!")
print("📋 Single Meal Results:")
print(meal_planner_result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6593e67b-2155-4871-99c1-3efcded969d1                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Task: Search for the best 'Kheer' recipe for 4 people within a ₹25 budget. Consider dietary restrictions:      │
│  ['no nuts'] and Taste preference : Sweet. Find recipes that match the Taste preference and provide complete    │
│  ingredient lists with quantities.                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Thought: Thought: The search results provide several Kheer recipes, all sweet and scalable for 4 servings,     │
│  nut-free by omitting nuts/raisins. Core ingredients are rice, milk, sugar, cardamom; quantities scaled down    │
│  from 1L milk (4-5 servings) to ~800ml for 4 people. Estimated India costs (2026): rice ₹4, milk ₹32, sugar     │
│  ₹3, cardamom ₹1 = ~₹40, but scale further and use minimal sugar/jaggery to fit ₹25 budget strictly. Use        │
│  recipe [1] base, adapt nut-free, provide researched ingredients list matching criteria. No full instructions   │
│  in schema, so list ingredients only in final JSON.                                                             │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

ERROR:root:OpenAI API call failed: Error code: 400 - {'error': {'message': 'Last message must have role `user` or `tool`.', 'type': 'invalid_message', 'code': 400}}
ERROR:root:OpenAI API call failed: Error code: 400 - {'error': {'message': 'Last message must have role `user` or `tool`.', 'type': 'invalid_message', 'code': 400}}


...

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 98310f94-282b-4134-8f46-c207a29db2a4                                                                     │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ Single meal planning completed!
📋 Single Meal Results:
meal_name='Rice Kheer (Chawal Ki Kheer) - No Nuts Version' taste='Sweet' servings=4 researched_ingredients=['¼ cup basmati rice (50 grams)', '1 liter whole milk (or 750ml for budget option)', '4-6 tablespoons sugar (50-65 grams)', '½ teaspoon green cardamom powder', '1 pinch saffron strands']


╭─────────────────────────────────────────── Trace Batch Finalization ────────────────────────────────────────────╮
│ ✅ Trace batch finalized with session ID: fc713ce8-c78e-4ecb-a5e2-809a341934de                                  │
│                                                                                                                 │
│ 🔗 View here:                                                                                                   │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/fc713ce8-c78e-4ecb-a5e2-809a341934de?access_code=TRA │
│ CE-82ada03482                                                                                                   │
│ 🔑 Access Code: TRACE-82ada03482                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6593e67b-2155-4871-99c1-3efcded969d1                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "meal_name": "Rice Kheer (Chawal Ki Kheer) - No Nuts Version",                                               │
│    "taste": "Sweet",                                                                                            │
│    "servings": 4,                                                                                               │
│    "researched_ingredients": [                                                                                  │
│      "¼ cup basmati rice (50 grams)",                                                                           │
│      "1 liter whole milk (or 750ml for budget option)",                                                         │
│      "4-6 tablespoons sugar (50-65 grams),                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [19]:
shopping_organizer = Agent(
    role="Shopping Organizer", 
    goal="Organize grocery lists by store sections efficiently",
    backstory="An experienced shopper who knows how to organize lists for quick store trips and considers dietary restrictions.",
    tools=[],
    llm=perplexity_llm,
    verbose=False
)

In [20]:
shopping_task = Task(
    description=(
        "Organize the ingredients from the '{meal_name}' meal plan into a grocery shopping list. "
        "Group items by store sections and estimate quantities for {servings} people. "
        "Consider dietary restrictions: {dietary_restrictions} and Taste Preferences: {taste}. "
        "Stay within budget: {budget}."
    ),
    expected_output="An organized shopping list grouped by store sections with quantities and prices.",
    agent=shopping_organizer,
    context=[meal_planning_task],
    output_pydantic=GroceryShoppingPlan,
    output_file="shopping_list.json"
)

### Building Our Two-Agent Grocery Crew

Now we're ready to create our first multi-agent workflow. This crew combines both the meal planner and shopping organizer to create a complete meal-to-shopping-list pipeline.

**How Multi-Agent Workflows Work:**
1. **Sequential Processing**: The `Process.sequential` ensures agents work in order - meal planning first, then shopping organization
2. **Data Flow**: The shopping organizer automatically receives the meal planner's output through the task context
3. **Structured Output**: The final result follows our `GroceryShoppingPlan` Pydantic model for consistent formatting

**Benefits of This Approach:**
- **Separation of Concerns**: Each agent has a specific role and expertise
- **Reusability**: We can swap out agents or add new ones without changing the overall workflow
- **Reliability**: Pydantic ensures our output is always properly structured


In [21]:
two_agent_grocery_crew = Crew(
    agents=[meal_planner, shopping_organizer],  # Both agents
    tasks=[meal_planning_task, shopping_task],   # Both tasks
    process=Process.sequential,
    verbose=True,
    memory = False,
)

# Run the complete crew (this will do BOTH meal planning AND shopping)
shopping_result = two_agent_grocery_crew.kickoff(
    inputs={
        "meal_name": "Kheer",
        "servings": 4,
        "budget": "₹25",                           
        "dietary_restrictions": ["no nuts"],      
        "taste": "sweet"               
    }
)

# Print the shopping results
print("✅ Complete meal planning + shopping completed!")
print("📋 Shopping Results:")
print(shopping_result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: fa60cb12-1585-4795-9dad-dd2e2e0e91c8                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Task: Search for the best 'Kheer' recipe for 4 people within a ₹25 budget. Consider dietary restrictions:      │
│  ['no nuts'] and Taste preference : sweet. Find recipes that match the Taste preference and provide complete    │
│  ingredient lists with quantities.                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Thought: Thought: I have been provided with search results about Kheer recipes. Let me analyze these results   │
│  to find the best recipe that matches the user's criteria: 4 servings, ₹25 budget, no nuts, and sweet taste     │
│  preference.                                                                                                    │
│  Looking at the search results:                                                                                 │
│  - All recipes provided contain nuts (almonds, cashews, pistachios), which conflicts with the dietary           │
│  restriction of "no nuts"                                                                                       │
│  - I need to search for a kheer recipe specifically without nuts to better match the user's requirements        │
│  - I should also verify budget feasibility for the ₹25 constraint                                               │
│  Let me search for a nut-free kheer recipe to provide a better match.                                           │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "search_query": "kheer recipe without nuts no nuts Indian rice pudding"                                      │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: OpenAI API call failed: Error code: 400 - {'error': {'message': 'Last message must have role `user` or  │
│  `tool`.', 'type': 'invalid_message', 'code': 400}}                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 98310f94-282b-4134-8f46-c207a29db2a4                                                                     │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: fa60cb12-1585-4795-9dad-dd2e2e0e91c8                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

BadRequestError: Error code: 400 - {'error': {'message': 'Last message must have role `user` or `tool`.', 'type': 'invalid_message', 'code': 400}}

In [ ]:
budget_advisor = Agent(
    role="Budget Advisor",
    goal="Provide cost estimates and money-saving tips",
    backstory="A budget-conscious shopper who helps families save money on groceries while respecting dietary needs.",
    tools=[SerperDevTool()],
    llm=perplexity_llm,
    verbose=False
)

### Defining the Budget Analysis Task

In [ ]:
budget_task = Task(
    description=(
        "Analyze the shopping plan for '{meal_name}' serving {servings} people. "
        "Ensure total cost stays within {budget}. Consider dietary restrictions: {dietary_restrictions}. "
        "Provide practical money-saving tips and alternative ingredients if needed to meet budget."
    ),
    expected_output="A complete shopping guide with detailed prices, budget analysis, and money-saving tips.",
    agent=budget_advisor,
    context=[meal_planning_task, shopping_task],
    output_file="shopping_guide.md"
)

In [ ]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/-5NXnHGkX1WG81tqE8kSTg/agents.yaml"

In [ ]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/7fhq1KPxi7cuqjOCBgcmXQ/tasks.yaml"

In [ ]:
%mkdir -p config
%mv agents.yaml  config/agents.yaml
%mv tasks.yaml   config/tasks.yaml

### Using CrewBase and Decorators with CrewAI

#### What is `@CrewBase`?
In CrewAI, `@CrewBase` is a Python class decorator that automates the collection and wiring of your agents and tasks — especially when you're organizing your crew logic in Python, YAML or hybrid config (YAML + code). Basically, when we have the following:

```python
from crewai.project import CrewBase
from crewai.project.annotations import agent, task

@CrewBase
class myCrew:
    ...
```
It tells CrewAI that: This class defines the agents, tasks, and crew I want to run. Please find them automatically and wire everything together.

You might think that we already defined separate Agent, Task and Crew why do we need CrewBase? We need it because this can get repetitive and hard to manage as your app grows. That’s where `@CrewBase` shines.

---

#### Why use `@CrewBase` + decorators?
- **Auto-discovers** methods decorated with @agent, @task, and @crew inside the class
- **Loads config** automatically from YAML (if present)
- Keeps everything **modular**, **scalable**, and **clean** for production setups

---

#### Decorators overview
- CrewAI provides several decorators that are used to mark methods within your crew class for special handling.
| Decorator           | Marks…                                            |
| :------------------ | :----------------------------------------------- |
| `@CrewBase`         | The class that holds your agents & tasks         |
| `@agent`            | A method that returns an `Agent` object          |
| `@task`             | A method that returns a `Task` object            |
| `@crew`             | (Optional) A method that returns a `Crew` object |
| `@before_kickoff`   | (Optional) Runs once **before** the crew starts  |
| `@after_kickoff`    | (Optional) Runs once **after** the crew finishes |

> The **CrewBase class**, along with these decorators, automates the collection of agents and tasks, reducing the need for manual management.


In [ ]:
from leftover import LeftoversCrew

leftovers_cb = LeftoversCrew(llm=perplexity_llm)
yaml_leftover_manager = leftovers_cb.leftover_manager()
yaml_leftover_task    = leftovers_cb.leftover_task()

In [ ]:
summary_agent = Agent(
    role="Report Compiler",
    goal="Compile comprehensive meal planning reports from all team outputs",
    backstory="A skilled coordinator who organizes information from multiple specialists into comprehensive, easy-to-follow reports.",
    tools=[],
    llm=perplexity_llm,
    verbose=False
)

In [ ]:
summary_task = Task(
    description=(
        "Compile a comprehensive meal planning report that includes:\n"
        "1. The complete recipe and cooking instructions from the meal planner\n"
        "2. The organized shopping list with prices from the shopping organizer\n"
        "3. The budget analysis and money-saving tips from the budget advisor\n"
        "4. The leftover management suggestions from the waste reduction specialist\n"
        "Format this as a complete, user-friendly meal planning guide."
    ),
    expected_output="A comprehensive meal planning guide that combines all team outputs into one cohesive report.",
    agent=summary_agent,
    context=[meal_planning_task, shopping_task, budget_task, yaml_leftover_task],
)

In [ ]:
complete_grocery_crew = Crew(
    agents=[
        meal_planner,           
        shopping_organizer,      
        budget_advisor,         
        yaml_leftover_manager,  # YAML-based leftover manager
        summary_agent           # New summary agent
    ],
    tasks=[
        meal_planning_task,     
        shopping_task,          
        budget_task,            
        yaml_leftover_task,    # YAML-based leftover task
        summary_task            # New summary task
    ],
    process=Process.sequential,
    verbose=True
)

In [ ]:
# Run the complete crew
complete_result = complete_grocery_crew.kickoff(
    inputs={
        "meal_name": "Kheer",
        "servings": 4,
        "budget": "₹25",
        "dietary_restrictions": ["no nuts", "low sodium"],
        "taste": "sweet"
    }
)

print("✅ Complete meal planning with summary completed!")
print("📋 Complete Results:")
print(complete_result)